In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_personel_columns
from lib.textract import parse_textract_datetime

In [2]:
pprr = pd.read_csv("../data/Harahan Civil Service Department/Administrative Data/Harahan_CSD_PPRR_2020.csv")
pprr.rename(columns=lambda x: x.strip(), inplace=True)
pprr.fillna("", inplace=True)

pat = re.compile(r"^([^\d]*)(\d{2}-\d{4})?([^\d]*)$")
pat2 = re.compile(r"^(.+, \w{2,}(?: \w)?) (\w{2,}.*)$")
rank_whitelist = set(['Captain', 'Police Officer', 'I/T Coordinator',
       'Assistant Chief Of Police', 'It Technician', 'Sergeant',
       'Jailer / Dispatch Provisional', 'Police Lieutenant',
       'Probational Police Officer', 'Secretary',
       'Jailer / Dispatch Probational', 'Police Mechanic', 'Police Sgt.',
       'Police Property Manager', 'Part Time Police Officer', 'Jailer',
       'Jailer/Dispatch', 'Provisional Sergeant',
       'Records Clerk', 'Police Captain', 'Police Chief Secretary',
       'Assist Chief Of Police', 'Police Dept. Agent',
       'Provisional Chief Secretary', 'Assistant Police Chief',
       'Police Records Clerk', 'Maintenance Man', 'Police Jailer',
       'Police Sgt', 'Mechanic', 'Police M/C Mechanic',
       'Dispatch / Jailer', 'Police Sergeant', 'Police Officer Sgt',
       'Police Chief', 'Secretary - Police Chief'])
record = dict()
records = []
for _, row in pprr.iterrows():
    match = pat.match(row[0])
    rank = (match.group(1) or "").title().strip()
    emp_no = match.group(2) or ""
    name = (match.group(3) or "").title().strip()
    if rank != "":
        record["rank"] = rank
        rank = ""
    if emp_no != "":
        if len(record) > 0:
            records.append(record)
            record = dict()
        record["emp_no"] = int(emp_no[3:])
    if name != "":
        record["name"] = name
        match2 = pat2.match(name)
        if match2 is not None and match2.group(2) in rank_whitelist:
            record["name"] = match2.group(1)
            record["rank"] = match2.group(2)
    if row[1] != "":
        record["dept_no"] = row[1].strip()
        record["annual"] = row[2]
        record["status"] = row[3]

if len(record) > 0:
    records.append(record)
    record = dict()

pprr = pd.DataFrame.from_records(records)
pprr.loc[23, "rank"] = "Police Officer"
pprr.loc[48, "rank"] = "Sergeant"
pprr.loc[73, "rank"] = "Jailer"
pprr.loc[98, "rank"] = "Police Officer"
pprr.set_index(["emp_no", "name"], inplace=True, verify_integrity=True)
pprr

,,dept_no,annual,status,rank
emp_no,name,,,,
2146,"Adams, Manuel J",010-112,"66,044.16",T,Captain
2308,"Bagley, Chase M",010-112,"40,906.32",T,Police Officer
2335,"Barnett, Michael",010-112,"40,513.20",A,Police Officer
2270,"Binder, Gregory J",010-112,"16,640.00",T,I/T Coordinator
2268,"Blackwell, Aaron J",010-112,"50,000.08",T,Assistant Chief Of Police
...,...,...,...,...,...
1082,"Walker, Robert T",010-112,"56,000.10",A,Police Chief
2301,"Ward, Carley A",010-112,"40,906.32",T,Police Officer
2278,"Whittle, Tracy M",010-112,"42,681.60",T,Secretary - Police Chief


In [3]:
payroll = pd.read_csv("../data/Harahan Civil Service Department/Administrative Data/Harahan_CSD_Payroll Report_2020_.csv")
payroll.rename(columns=lambda x: x.strip(), inplace=True)
payroll.iloc[:, 0] = payroll.iloc[:, 0].fillna(0).astype("int32")
payroll.iloc[:, 1] = payroll.iloc[:, 1].str.replace(". . ", "., ", regex=False)\
    .str.strip().str.title()

records = []
record = dict()
for _, row in payroll.iterrows():
    if row[0] != 0:
        record["emp_no"] = row[0]
        record["name"] = row[1]
    else:
        record["emp_no"] = row[1][:4]
        record["name"] = row[1][5:]
    record["status"] = row[2]
    record["p_f"] = row[3][0]
    record["dept_no"] = row[3][2:].strip()
    record["hire"] = row[4]
    record["termination"] = row[5]
    records.append(record)
    record = dict()
payroll = pd.DataFrame.from_records(records)
payroll.drop_duplicates(inplace=True)
payroll.set_index(["emp_no", "name"], inplace=True, verify_integrity=True)

payroll

,,status,p_f,dept_no,hire,termination
emp_no,name,,,,,
2146,"Adams, Manuel J",T,F,010-112,1/2/02,6/29/20
2308,"Bagley, Chase M",T,F,010-112,2/24/19,10/16/20
2335,"Barnett, Michael",A,F,010-112,2/17/20,NaN
2270,"Binder, Gregory J",T,F,010-112,12/4/13,NaN
2268,"Blackwell, Aaron J",T,E,010-112,11/3/13,5/9/14
...,...,...,...,...,...,...
2278,"Whittle, Tracy M",T,F,010-112,6/2/14,5/29/19
2287,"Wright, Michael P",T,F,010-112,8/2/15,12/19/18
2205,"Yeadon, Andrew D",T,F,010-112,4/2/08,1/29/09


In [4]:
joined = payroll.join(pprr[["annual", "rank"]])
joined

,,status,p_f,dept_no,hire,termination,annual,rank
emp_no,name,,,,,,,
2146,"Adams, Manuel J",T,F,010-112,1/2/02,6/29/20,"66,044.16",Captain
2308,"Bagley, Chase M",T,F,010-112,2/24/19,10/16/20,"40,906.32",Police Officer
2335,"Barnett, Michael",A,F,010-112,2/17/20,NaN,"40,513.20",Police Officer
2270,"Binder, Gregory J",T,F,010-112,12/4/13,NaN,"16,640.00",I/T Coordinator
2268,"Blackwell, Aaron J",T,E,010-112,11/3/13,5/9/14,"50,000.08",Assistant Chief Of Police
...,...,...,...,...,...,...,...,...
2278,"Whittle, Tracy M",T,F,010-112,6/2/14,5/29/19,NaN,NaN
2287,"Wright, Michael P",T,F,010-112,8/2/15,12/19/18,NaN,NaN
2205,"Yeadon, Andrew D",T,F,010-112,4/2/08,1/29/09,NaN,NaN


In [5]:
df = joined[["dept_no", "hire", "termination", "annual", "rank"]]
df.reset_index(inplace=True)

names = df["name"].str.replace(". . ", "., ", regex=False)\
    .str.rsplit(", ", n=1, expand=True)
df.loc[:, "Last Name"] = names.iloc[:, 0].str.replace(", ", " ", regex=False)\
    .str.replace("(.+)(Ii|Iii|Iv)$", lambda m: m.group(1) + m.group(2).upper())
names = names.iloc[:, 1].str.split(" ", expand=True)
df.loc[:, "First Name"] = names.iloc[:, 0]
df.loc[:, "Middle Name"] = names.iloc[:, 1]\
    .where(names.iloc[:, 1].notnull(), np.NaN)

df.rename(columns={
    "dept_no": "Department #",
    "hire": "Hire Date",
    "termination": "Termination Date",
    "annual": "Annual Salary",
    "rank": "Rank",
    "emp_no": "Employee ID #",
}, inplace=True)

df.loc[:, "Annual Salary"] = df["Annual Salary"]\
    .str.replace(",", "", regex=False).astype("float64")
df.loc[:, "Hire Date"] = parse_textract_datetime(df["Hire Date"])
df.loc[:, "Termination Date"] = parse_textract_datetime(df["Termination Date"])
df = rearrange_personel_columns(df)
df

,Last Name,Middle Name,First Name,Employee ID #,Department #,Rank,Hire Date,Termination Date,Annual Salary
0,Adams,J,Manuel,2146,010-112,Captain,2002-01-02,2020-06-29,66044.16
1,Bagley,M,Chase,2308,010-112,Police Officer,2019-02-24,2020-10-16,40906.32
2,Barnett,NaN,Michael,2335,010-112,Police Officer,2020-02-17,NaT,40513.20
3,Binder,J,Gregory,2270,010-112,I/T Coordinator,2013-12-04,NaT,16640.00
4,Blackwell,J,Aaron,2268,010-112,Assistant Chief Of Police,2013-11-03,2014-05-09,50000.08
...,...,...,...,...,...,...,...,...,...
121,Whittle,M,Tracy,2278,010-112,NaN,2014-06-02,2019-05-29,NaN
122,Wright,P,Michael,2287,010-112,NaN,2015-08-02,2018-12-19,NaN
123,Yeadon,D,Andrew,2205,010-112,NaN,2008-04-02,2009-01-29,NaN
124,Yeadon,M,Robert,2199,010-112,NaN,2007-07-06,2008-08-02,NaN


In [6]:
df.to_csv("../output/harahan-csd_personel_2020.csv", index=False)